***GENERATED CODE FOR retailcreditscoring PIPELINE***
***DON'T EDIT THIS CODE***

**CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM**

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


**TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA**

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


**AUTOML FUNCTIONS**

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = df.drop(label, axis=1).values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': sparkDF.columns}
    return data


**READING DATAFRAME**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

%run retailcreditscoringHooks.ipynb
try:
	sourcePreExecutionHook()

	retailcreditscoringdbfs = DBFSConnector.fetch(spark, "{'url': '/Demo/RetailCreditScoringTrain.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")

	sourcePostExecutionHook(retailcreditscoringdbfs)

except Exception as ex: 
	logging.error(ex)


**TRANSFORMING DATAFRAME**

In [ ]:
%run retailcreditscoringHooks.ipynb
try:
	transformationPreExecutionHook()

	retailcreditscoringautofe = TransformationMain.run(retailcreditscoringdbfs,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Creditability", "std_scalar": {"mean_flag": "True", "std_flag": "False"}, "transformation_label": "novalue"}], "feature": "Creditability", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "274", "mean": "0.75", "stddev": "0.43", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Account_Balance", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "274", "mean": "2.68", "stddev": "1.27", "min": "1", "max": "4", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Duration_of_Credit(month)", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "274", "mean": "20.89", "stddev": "12.83", "min": "4", "max": "60", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Payment_Status_of_Previous_Credit", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "274", "mean": "2.58", "stddev": "1.1", "min": "0", "max": "4", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Purpose", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "274", "mean": "2.6", "stddev": "2.64", "min": "0", "max": "10", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Credit_Amount", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "274", "mean": "3417.81", "stddev": "3038.6", "min": "362", "max": "15653", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Value_Savings_Stocks", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "274", "mean": "2.11", "stddev": "1.6", "min": "1", "max": "5", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Length_of_current_employment", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "274", "mean": "3.43", "stddev": "1.18", "min": "1", "max": "5", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Instalment_per_cent", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "274", "mean": "2.99", "stddev": "1.15", "min": "1", "max": "4", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Sex_&_Marital_Status", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "274", "mean": "2.68", "stddev": "0.69", "min": "1", "max": "4", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Guarantors", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "274", "mean": "1.11", "stddev": "0.43", "min": "1", "max": "3", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Duration_in_Current_address", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "274", "mean": "2.8", "stddev": "1.13", "min": "1", "max": "4", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Most_valuable_available_asset", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "274", "mean": "2.33", "stddev": "1.03", "min": "1", "max": "4", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age(years)", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "274", "mean": "35.45", "stddev": "10.94", "min": "19", "max": "74", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Concurrent_Credits", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "274", "mean": "2.68", "stddev": "0.71", "min": "1", "max": "3", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Type_of_apartment", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "274", "mean": "1.95", "stddev": "0.51", "min": "1", "max": "3", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "No_of_Credits_at_this_Bank", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "274", "mean": "1.44", "stddev": "0.61", "min": "1", "max": "4", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Occupation", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "274", "mean": "2.91", "stddev": "0.67", "min": "1", "max": "4", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "No_of_dependents", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "274", "mean": "1.17", "stddev": "0.37", "min": "1", "max": "2", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Telephone", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "274", "mean": "1.42", "stddev": "0.49", "min": "1", "max": "2", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Foreign_Worker", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "274", "mean": "1.04", "stddev": "0.21", "min": "1", "max": "2", "missing": "0"}}]}))

	transformationPostExecutionHook(retailcreditscoringautofe)

except Exception as ex: 
	logging.error(ex)


**TRAIN MODEL**

In [ ]:
%run retailcreditscoringHooks.ipynb
try:
	mlPreExecutionHook()

	dataAutoML=functionClassification(retailcreditscoringautofe, ["Account_Balance", "Duration_of_Credit(month)", "Payment_Status_of_Previous_Credit", "Purpose", "Credit_Amount", "Value_Savings_Stocks", "Length_of_current_employment", "Instalment_per_cent", "Sex_&_Marital_Status", "Guarantors", "Duration_in_Current_address", "Most_valuable_available_asset", "Age(years)", "Concurrent_Credits", "Type_of_apartment", "No_of_Credits_at_this_Bank", "Occupation", "No_of_dependents", "Telephone", "Foreign_Worker"], "Creditability")

	mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


**PREDICT ON TRAINED MODEL**

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

